In [1]:
import pandas as pd
import scipy.stats as stats
import cptac.utils as ut
import scipy
import statsmodels.stats.multitest as ssm
import numpy as np
import sys, os
currentdir = os.path.dirname(os.path.realpath('make_regression_df.py'))
parentdir = os.path.dirname(currentdir)
sys.path.append(parentdir)
import pcprutils as dc

In [2]:
cancers, cancer_names = dc.load_cancers()

/Users/humbertogiraldez/opt/anaconda3/lib/python3.9/site-packages/gtfparse/read_gtf.py:82: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  chunk_iterator = pd.read_csv(
/Users/humbertogiraldez/opt/anaconda3/lib/python3.9/site-packages/gtfparse/read_gtf.py:82: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  chunk_iterator = pd.read_csv(


/Users/humbertogiraldez/opt/anaconda3/lib/python3.9/site-packages/cptac/pancan/washuccrcc.py:136: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_df = tumor.append(normal)
/Users/humbertogiraldez/opt/anaconda3/lib/python3.9/site-packages/cptac/pancan/washuccrcc.py:136: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_df = tumor.append(normal)
/Users/humbertogiraldez/opt/anaconda3/lib/python3.9/site-packages/cptac/pancan/washuccrcc.py:136: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_df = tumor.append(normal)


/Users/humbertogiraldez/opt/anaconda3/lib/python3.9/site-packages/gtfparse/read_gtf.py:82: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  chunk_iterator = pd.read_csv(
/Users/humbertogiraldez/opt/anaconda3/lib/python3.9/site-packages/gtfparse/read_gtf.py:82: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  chunk_iterator = pd.read_csv(


/Users/humbertogiraldez/opt/anaconda3/lib/python3.9/site-packages/cptac/pancan/washuccrcc.py:215: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rna_combined = rna_tumor.append(rna_normal)


/Users/humbertogiraldez/opt/anaconda3/lib/python3.9/site-packages/gtfparse/read_gtf.py:82: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  chunk_iterator = pd.read_csv(
/Users/humbertogiraldez/opt/anaconda3/lib/python3.9/site-packages/gtfparse/read_gtf.py:82: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  chunk_iterator = pd.read_csv(


...

/Users/humbertogiraldez/opt/anaconda3/lib/python3.9/site-packages/cptac/pancan/washuhnscc.py:222: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rna_combined = rna_tumor.append(rna_normal)


/Users/humbertogiraldez/opt/anaconda3/lib/python3.9/site-packages/gtfparse/read_gtf.py:82: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  chunk_iterator = pd.read_csv(
/Users/humbertogiraldez/opt/anaconda3/lib/python3.9/site-packages/gtfparse/read_gtf.py:82: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  chunk_iterator = pd.read_csv(


/Users/humbertogiraldez/opt/anaconda3/lib/python3.9/site-packages/cptac/pancan/washulscc.py:142: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_df = tumor.append(normal)
/Users/humbertogiraldez/opt/anaconda3/lib/python3.9/site-packages/cptac/pancan/washulscc.py:142: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_df = tumor.append(normal)
/Users/humbertogiraldez/opt/anaconda3/lib/python3.9/site-packages/cptac/pancan/washulscc.py:142: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_df = tumor.append(normal)


/Users/humbertogiraldez/opt/anaconda3/lib/python3.9/site-packages/gtfparse/read_gtf.py:82: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  chunk_iterator = pd.read_csv(
/Users/humbertogiraldez/opt/anaconda3/lib/python3.9/site-packages/gtfparse/read_gtf.py:82: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  chunk_iterator = pd.read_csv(


/Users/humbertogiraldez/opt/anaconda3/lib/python3.9/site-packages/cptac/pancan/washulscc.py:221: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rna_combined = rna_tumor.append(rna_normal)


In [3]:
def log2_fold_change(tumor, normal):
    tumor = np.mean(tumor)
    normal = np.mean(normal)
    if normal == 0 or tumor == 0:
        return(float('Nan'))
    fold_change = tumor / normal
    fold_change = abs(fold_change)
    log2_change = np.log2(fold_change)
    return log2_change    

In [4]:
diff_expression_dfs = []
for cancer, cancer_name in zip(cancers, cancer_names):
    prot_df = dc.get_prot_trans_df(cancer)
    prot_df = prot_df.drop(columns = 'Transcriptomics')
    fold_change = prot_df.groupby('Gene').apply(
        lambda df: log2_fold_change(df[df['Tissue']=='Tumor'].Proteomics,
                                    df[df['Tissue']=='Normal'].Proteomics))
    fold_change = pd.DataFrame(fold_change, columns = ['Log2_fold_change'])
    fold_change.reset_index(inplace = True)
    ranksums = prot_df.groupby('Gene').apply(lambda df: stats.ranksums(
        df[df['Tissue']=='Tumor'].Proteomics,
        df[df['Tissue']=='Normal'].Proteomics))
    ranksums = pd.DataFrame.from_records(ranksums, index = ranksums.index, columns = ['statistic', 'pval'])
    ranksums.reset_index(inplace = True)
    ranksums = ranksums.dropna()
    ranksums['FDR'] = ssm.fdrcorrection(ranksums['pval'])[1]
    ranksums['Cancer'] = [cancer_name] * len(ranksums)
    diff_expression_df = pd.merge(ranksums, fold_change)
    diff_expression_dfs.append(diff_expression_df)
diff_expression_df = pd.concat(diff_expression_dfs)
diff_expression_df.to_csv('Proteomics_differential_expression_df.csv', index = False)
diff_expression_df

/Users/humbertogiraldez/opt/anaconda3/lib/python3.9/site-packages/scipy/stats/stats.py:7784: RuntimeWarning: invalid value encountered in double_scalars
  z = (s - expected) / np.sqrt(n1*n2*(n1+n2+1)/12.0)
/Users/humbertogiraldez/opt/anaconda3/lib/python3.9/site-packages/scipy/stats/stats.py:7784: RuntimeWarning: invalid value encountered in double_scalars
  z = (s - expected) / np.sqrt(n1*n2*(n1+n2+1)/12.0)
/Users/humbertogiraldez/opt/anaconda3/lib/python3.9/site-packages/scipy/stats/stats.py:7784: RuntimeWarning: invalid value encountered in double_scalars
  z = (s - expected) / np.sqrt(n1*n2*(n1+n2+1)/12.0)
/Users/humbertogiraldez/opt/anaconda3/lib/python3.9/site-packages/scipy/stats/stats.py:7784: RuntimeWarning: invalid value encountered in double_scalars
  z = (s - expected) / np.sqrt(n1*n2*(n1+n2+1)/12.0)


,Gene,statistic,pval,FDR,Cancer,Log2_fold_change
0,A1BG,3.182361,1.460796e-03,2.326141e-03,CCRCC,0.146002
1,A1CF,-6.023555,1.706272e-09,4.312462e-09,CCRCC,1.200982
2,A2M,5.044797,4.540027e-07,9.714156e-07,CCRCC,-1.418060
3,A4GALT,0.979796,3.271869e-01,3.727230e-01,CCRCC,4.510855
4,AAAS,8.283087,1.200225e-16,4.871209e-16,CCRCC,1.557699
...,...,...,...,...,...,...
12177,ZWINT,9.762358,1.633135e-22,6.411490e-22,LSCC,-1.726952
12178,ZXDC,9.652270,4.807927e-22,1.838938e-21,LSCC,-1.545166
12179,ZYG11B,-8.743083,2.268343e-18,7.072680e-18,LSCC,-2.893608
12180,ZYX,-11.679922,1.614446e-31,1.542524e-30,LSCC,-0.454489


In [5]:
diff_expression_dfs = []
for cancer, cancer_name in zip(cancers, cancer_names):
    prot_df = dc.get_prot_trans_df(cancer)
    prot_df = prot_df.drop(columns = 'Proteomics')
    fold_change = prot_df.groupby('Gene').apply(
        lambda df: log2_fold_change(df[df['Tissue']=='Tumor'].Transcriptomics,
                                    df[df['Tissue']=='Normal'].Transcriptomics))
    fold_change = pd.DataFrame(fold_change, columns = ['Log2_fold_change'])
    fold_change.reset_index(inplace = True)
    ranksums = prot_df.groupby('Gene').apply(lambda df: stats.ranksums(
        df[df['Tissue']=='Tumor'].Transcriptomics,
        df[df['Tissue']=='Normal'].Transcriptomics))
    ranksums = pd.DataFrame.from_records(ranksums, index = ranksums.index, columns = ['statistic', 'pval'])
    ranksums.reset_index(inplace = True)
    ranksums = ranksums.dropna()
    ranksums['FDR'] = ssm.fdrcorrection(ranksums['pval'])[1]
    ranksums['Cancer'] = [cancer_name] * len(ranksums)
    diff_expression_df = pd.merge(ranksums, fold_change)
    diff_expression_dfs.append(diff_expression_df)
diff_expression_df = pd.concat(diff_expression_dfs)
diff_expression_df.to_csv('Transcriptomics_differential_expression_df.csv', index = False)
diff_expression_df

/Users/humbertogiraldez/opt/anaconda3/lib/python3.9/site-packages/scipy/stats/stats.py:7784: RuntimeWarning: invalid value encountered in double_scalars
  z = (s - expected) / np.sqrt(n1*n2*(n1+n2+1)/12.0)
/Users/humbertogiraldez/opt/anaconda3/lib/python3.9/site-packages/scipy/stats/stats.py:7784: RuntimeWarning: invalid value encountered in double_scalars
  z = (s - expected) / np.sqrt(n1*n2*(n1+n2+1)/12.0)
/Users/humbertogiraldez/opt/anaconda3/lib/python3.9/site-packages/scipy/stats/stats.py:7784: RuntimeWarning: invalid value encountered in double_scalars
  z = (s - expected) / np.sqrt(n1*n2*(n1+n2+1)/12.0)
/Users/humbertogiraldez/opt/anaconda3/lib/python3.9/site-packages/scipy/stats/stats.py:7784: RuntimeWarning: invalid value encountered in double_scalars
  z = (s - expected) / np.sqrt(n1*n2*(n1+n2+1)/12.0)


,Gene,statistic,pval,FDR,Cancer,Log2_fold_change
0,A1BG,-1.848454,6.453670e-02,7.917062e-02,CCRCC,-0.214259
1,A1CF,-5.257327,1.461639e-07,2.877988e-07,CCRCC,-0.640548
2,A2M,5.570530,2.539657e-08,5.259039e-08,CCRCC,0.498036
3,A4GALT,0.244949,8.064959e-01,8.249319e-01,CCRCC,0.077638
4,AAAS,0.906050,3.649094e-01,4.001595e-01,CCRCC,0.040925
...,...,...,...,...,...,...
12177,ZWINT,11.807520,3.569340e-32,5.226166e-31,LSCC,2.851481
12178,ZXDC,0.351118,7.254995e-01,7.515335e-01,LSCC,0.118920
12179,ZYG11B,-8.536132,1.387896e-17,4.400664e-17,LSCC,-0.401381
12180,ZYX,-9.359284,8.027877e-21,3.160815e-20,LSCC,-0.744325
